In [1]:
"""Simple Pickup Delivery Problem (PDP)."""

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ]
    data['pickups_deliveries'] = [
        [1, 6],
        [2, 10],
        [4, 3],
        [5, 9],
        [7, 8],
        [15, 11],
        [13, 12],
        [16, 14],
    ]
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, assignment):
    """Prints assignment on console."""
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        total_distance += route_distance
    print('Total Distance of all routes: {}m'.format(total_distance))


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Define Transportation Requests.
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(
                delivery_index))
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index) <=
            distance_dimension.CumulVar(delivery_index))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if assignment:
        print_solution(data, manager, routing, assignment)


if __name__ == '__main__':
    main()

Route for vehicle 0:
 0 ->  13 ->  15 ->  11 ->  12 -> 0
Distance of the route: 1552m

Route for vehicle 1:
 0 ->  5 ->  2 ->  10 ->  16 ->  14 ->  9 -> 0
Distance of the route: 2192m

Route for vehicle 2:
 0 ->  4 ->  3 -> 0
Distance of the route: 1392m

Route for vehicle 3:
 0 ->  7 ->  1 ->  6 ->  8 -> 0
Distance of the route: 1780m

Total Distance of all routes: 6916m


In [2]:
import json
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm_notebook
%matplotlib inline

def load_data(file):
    """Загрузка входных данных из файла"""
    with open(file, 'r') as f:
        input_data = json.load(f)
    couriers = {}
    orders = {}
    points = {}
    for depotData in input_data['depots']:
        points[depotData['point_id']] = {
            'location': [depotData['location_x'], depotData['location_y']],
            'timewindow': [0, 1439],
        }
    for courierData in input_data['couriers']:
        couriers[courierData['courier_id']] = {
            'location': [courierData['location_x'], courierData['location_y']],
            'time': 360,
        }
    for orderData in input_data['orders']:
        points[orderData['pickup_point_id']] = {
            'location': [orderData['pickup_location_x'], orderData['pickup_location_y']],
            'timewindow': [orderData['pickup_from'], orderData['pickup_to']],
            'order_time': {orderData['order_id']: orderData['pickup_from']}
        }
        points[orderData['dropoff_point_id']] = {
            'location': [orderData['dropoff_location_x'], orderData['dropoff_location_y']],
            'timewindow': [orderData['dropoff_from'], orderData['dropoff_to']],
        }
        orders[orderData['order_id']] = orderData
    return couriers, orders, points

def get_travel_duration_minutes(location1, location2):
    """Время перемещения курьера от точки location1 до точки location2 вминутах"""
    distance = abs(location1[0] - location2[0]) + abs(location1[1] - location2[1])
    return 10 + distance

def get_current_courier_reward(time, payment):
    work_payment = time * 2
    profit = orders_payment - work_payment
    return profit

def delete_element(array, idx):
    result = []
    for i in range(len(array)):
        if not(i == idx):
            result.append(array[i])
    return result

In [159]:
data

({1: {'location': [20, 20], 'time': 360},
  2: {'location': [50, 50], 'time': 360},
  3: {'location': [150, 20], 'time': 360},
  4: {'location': [25, 150], 'time': 360},
  5: {'location': [100, 300], 'time': 360}},
 {10001: {'order_id': 10001,
   'pickup_point_id': 40001,
   'pickup_location_x': 20,
   'pickup_location_y': 100,
   'pickup_from': 360,
   'pickup_to': 1380,
   'dropoff_point_id': 60001,
   'dropoff_location_x': 60,
   'dropoff_location_y': 110,
   'dropoff_from': 360,
   'dropoff_to': 1180,
   'payment': 900},
  10002: {'order_id': 10002,
   'pickup_point_id': 40002,
   'pickup_location_x': 55,
   'pickup_location_y': 100,
   'pickup_from': 360,
   'pickup_to': 1380,
   'dropoff_point_id': 60002,
   'dropoff_location_x': 63,
   'dropoff_location_y': 130,
   'dropoff_from': 560,
   'dropoff_to': 1380,
   'payment': 900},
  10003: {'order_id': 10003,
   'pickup_point_id': 40003,
   'pickup_location_x': 16,
   'pickup_location_y': 104,
   'pickup_from': 420,
   'pickup_to':

In [210]:
data = load_data("data/contest_input.json")

In [211]:
couriers_start_position = [data[0][i+1]['location'] for i in range(len(data[0]))]
points_pickup = np.array([[data[1][i+10001]['pickup_location_x'],
                           data[1][i+10001]['pickup_location_y']] for i in range(len(data[1]))])
time_pickup = np.array([[data[1][10001+i]['pickup_from'],
                         data[1][10001+i]['pickup_to']] for i in range(len(data[1]))])
time_dropoff = np.array([[data[1][10001+i]['dropoff_from'],
                          data[1][10001+i]['dropoff_to']] for i in range(len(data[1]))])    
points_dropoff = np.array([[data[1][i+10001]['dropoff_location_x'],
                            data[1][i+10001]['dropoff_location_y']] for i in range(len(data[1]))])
payments = np.array([data[1][10001+i]['payment'] for i in range(len(data[1]))])
pickup_id = np.array([data[1][10001+i]['pickup_point_id'] for i in range(len(data[1]))])
dropoff_id = np.array([data[1][10001+i]['dropoff_point_id'] for i in range(len(data[1]))])

In [212]:
len(data[1])

7303

In [213]:
from tqdm import tqdm_notebook

def get_by_id(point_id):
    if point_id < 10000:
        return couriers_start_position[point_id]
    if point_id < 60000:
        return points_pickup[point_id-40001]
    else:
        return points_dropoff[point_id-60001]

def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 0)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value
    return vector 


def get_directions_table(pickup_ids):
    directions_table = []
    for pickup_id in pickup_ids:
        n = pickup_id - 40001
        n_pickup_in_table = len(data[0]) + n
        n_dropoff_in_table = len(data[0]) + len(data[1]) + n 
        directions_table.append([n_pickup_in_table, n_dropoff_in_table])
    return directions_table

def create_distance_matrix_and_distance_table(pickup_id, dropoff_id, n_couriers):
    tmp_string = np.concatenate((np.array(range(n_couriers)), 
                                 pickup_id, 
                                 dropoff_id))
    n_pickup = len(data[1])
    n_dropoff = len(data[1])
    matrix = []
    for i in tqdm_notebook(range(len(tmp_string))):
        tmp = []
        for j in range(len(tmp_string)):
            tmp.append(get_travel_duration_minutes(get_by_id(tmp_string[j]), 
                                                   get_by_id(tmp_string[i])))
            
            
        matrix.append(tmp)
    return np.pad(matrix, 1, pad_with)[:-1,:-1]

In [214]:
dist = get_directions_table(pickup_id)

In [ ]:
matrix = create_distance_matrix(pickup_id, dropoff_id, len(data[0]))

In [ ]:
dist

In [ ]:
n_couriers = len(data[0])

In [208]:
def create_data_model(matrix, dist, n_couriers):
    d = {'distance_matrix': matrix, 
         'pickups_deliveries':dist,
        'num_vehicles':n_couriers,
        'depot':0}
    return d
    # [END data_model]


In [209]:
def print_solution(data, manager, routing, assignment):
    """Prints assignment on console."""
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        total_distance += route_distance
    print('Total Distance of all routes: {}m'.format(total_distance))
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model(matrix, dist, n_couriers)
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)

    # [END routing_model]

    # Define cost of each arc.
    # [START arc_cost]
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Add Distance constraint.
    # [START distance_constraint]
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    # [END distance_constraint]

    # Define Transportation Requests.
    # [START pickup_delivery_constraint]
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(
                delivery_index))
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index) <=
            distance_dimension.CumulVar(delivery_index))
    # [END pickup_delivery_constraint]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    assignment = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if assignment:
        print_solution(data, manager, routing, assignment)
    # [END print_solution]


if __name__ == '__main__':
    main()
# [END program]

Route for vehicle 0:
 0 ->  8 ->  6 ->  7 ->  11 ->  12 ->  13 ->  4 -> 0
Distance of the route: 207m

Route for vehicle 1:
 0 ->  5 ->  10 -> 0
Distance of the route: 336m

Route for vehicle 2:
 0 ->  2 ->  9 ->  14 ->  15 -> 0
Distance of the route: 201m

Route for vehicle 3:
 0 ->  1 -> 0
Distance of the route: 0m

Route for vehicle 4:
 0 ->  3 -> 0
Distance of the route: 0m

Total Distance of all routes: 744m
